In [11]:
!pip install simpy

### Goal
- feature out Simpy events, process and timeout

In [1]:
from simpy import Environment, Process

In [5]:
class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now:2.1f}") 
        duration = len(self.name)
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now:2.1f}") 

In [17]:
class VisitorSource(Process):  
    def __init__(self, env, names):
        super().__init__(env, self.schedule())
        self.names = names

    def schedule(self): 
        for name in self.names:
            yield env.timeout(1)
            print(f"start {name} at {self.env.now:2.1f}") 
            Visitor(self.env, name)

In [18]:
sim_duration = 10
env = Environment()
source = VisitorSource(env, ["Alice", "Bob"])
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')

start at 0
start Alice at 1.0
Alice: Here I am at 1.0
start Bob at 2.0
Bob: Here I am at 2.0
Bob: I must leave at 5.0
Alice: I must leave at 6.0
stop at 10


please note that visitor whow once and leave

events.Event
|
+— events.Timeout
|
+— events.Initialize
|
+— events.Process
|
+— events.Condition
|  |
|  +— events.AllOf
|  |
|  +— events.AnyOf
.
.

generator is an iterator, but it easier to write (example with __iter__ and __next__)
as it is an iterator ou may user next


now add a callback
never -processed but automaticaly triggered

run sim until event occur

In [17]:
def my_callback(timeout):
    print(f'Called back from {timeout}')
    print(timeout)
    print(f"now {timeout.env.now}")
    print(f"timeout {timeout}")
    print(f"timeout value {timeout.value}")
    print(f"triggered {timeout.triggered}")
    print(f"processed {timeout.processed}")

env = Environment()
visited = Visitor(env, "Alice")
visited.callbacks.append(my_callback)
env.run(until=visited)
print(f'stop at {env.now}')

Alice: Here I am at 0.0
Alice: I must leave at 5.0
Called back from <Visitor(visit) object at 0x1082dca30>
<Visitor(visit) object at 0x1082dca30>
now 5
timeout <Visitor(visit) object at 0x1082dca30>
timeout value None
triggered True
processed True
stop at 5


until n stands for until timeout(n)
stops when the process is done

until expect any event or condition

run sim until event occur
event.succeed
happy with visits

condition and multiple condition

In [25]:
class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name
        self.startAt = env.now

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now:2.1f}") 
        duration = len(self.name)
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now:2.1f}") 

In [35]:
class VisitorMonitor():  
    def __init__(self):
        self.data = []

    def trace(self, event): 
        print(f'Called back from {event} at {event.env.now:2.1f}')
        self.data.append({
            'name':event.name, 
            'startAt':event.startAt, 
            'stopAt':event.env.now 
        })
        
    def collect(self):
        return self.data

In [33]:
class VisitorSource(Process):  
    def __init__(self, env, names):
        super().__init__(env, self.schedule())
        self.names = names
        self.monitor = VisitorMonitor()

    def schedule(self): 
        for name in self.names:
            yield env.timeout(1)
            print(f"start {name} at {self.env.now:2.1f}") 
            visitor = Visitor(self.env, name)
            visitor.callbacks.append(self.monitor.trace)

In [36]:
sim_duration = 10
env = Environment()
source = VisitorSource(env, ["Alice", "Bob"])
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')
display(source.monitor.collect())

start at 0
start Alice at 1.0
Alice: Here I am at 1.0
start Bob at 2.0
Bob: Here I am at 2.0
Bob: I must leave at 5.0
Called back from <Visitor(visit) object at 0x111ca5520> at 5.0
Alice: I must leave at 6.0
Called back from <Visitor(visit) object at 0x111ca5be0> at 6.0
stop at 10


[{'name': 'Bob', 'startAt': 2, 'stopAt': 5},
 {'name': 'Alice', 'startAt': 1, 'stopAt': 6}]